<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/adam_area_perimetr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# 1. Генерация обучающих данных (10 примеров)
np.random.seed(42)  # Для воспроизводимости результатов
length = np.random.uniform(1, 10, size=100)  # Длина от 1 до 10
width = np.random.uniform(1, 10, size=100)   # Ширина от 1 до 10

# Целевые значения: площадь и периметр
area = length * width
perimeter = 2 * (length + width)

# Объединяем длину и ширину в одну матрицу признаков X
# Добавляем признак x^0 (единицы) для удобства
X = np.column_stack([length, width])
print("Обучающие данные (длина, ширина):")
print(X)
print("\nПлощадь (Y_area):", area)
print("Периметр (Y_perim):", perimeter)

# 2. Создаем полиномиальные признаки 3-й степени
def create_poly_features(X, degree=3):
    """Создает полиномиальные признаки для многомерного ввода"""
    n_samples, n_features = X.shape
    features = [np.ones(n_samples)]  # Добавляем единицы (bias term)

    # Добавляем оригинальные признаки
    for d in range(1, degree + 1):
        for i in range(n_features):
            features.append(X[:, i] ** d)

    # Добавляем взаимодействия признаков (например, length*width)
    for i in range(n_features):
        for j in range(i + 1, n_features):
            features.append(X[:, i] * X[:, j])

    return np.column_stack(features)

# Создаем расширенную матрицу признаков
X_poly = create_poly_features(X, degree=3)
print(f"\nРазмерность полиномиальных признаков: {X_poly.shape}")

# 3. Инициализация параметров модели
def initialize_parameters(n_features):
    """Инициализирует веса маленькими случайными значениями"""
    return np.random.randn(n_features) * 0.01

# 4. Реализация оптимизатора Adam
class AdamOptimizer:
    def __init__(self, params, lr=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = np.zeros_like(params)  # Первый момент
        self.v = np.zeros_like(params)  # Второй момент
        self.t = 0  # Счетчик итераций

    def update(self, params, grads):
        """Обновляет параметры с помощью алгоритма Adam"""
        self.t += 1
        self.m = self.beta1 * self.m + (1 - self.beta1) * grads
        self.v = self.beta2 * self.v + (1 - self.beta2) * (grads ** 2)

        # Коррекция bias
        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)

        # Обновление параметров
        params -= self.lr * m_hat / (np.sqrt(v_hat) + self.epsilon)
        return params

# 5. Функции для обучения
def compute_loss(y_true, y_pred):
    """Вычисляет среднеквадратичную ошибку"""
    return np.mean((y_true - y_pred) ** 2)

def compute_gradients(X, y_true, y_pred):
    """Вычисляет градиенты для линейной регрессии"""
    error = y_pred - y_true
    gradients = np.dot(X.T, error) / len(y_true)
    return gradients

def train_model(X, y, n_epochs=1000, lr=0.01):
    """Обучает модель для одного целевого значения"""
    n_features = X.shape[1]
    params = initialize_parameters(n_features)
    optimizer = AdamOptimizer(params, lr=lr)

    losses = []
    for epoch in range(n_epochs):
        # Прямое распространение
        y_pred = np.dot(X, params)

        # Вычисление потерь
        loss = compute_loss(y, y_pred)
        losses.append(loss)

        # Обратное распространение
        grads = compute_gradients(X, y, y_pred)

        # Обновление параметров
        params = optimizer.update(params, grads)

        if epoch % 200 == 0:
            print(f"Эпоха {epoch}, Loss: {loss:.6f}")

    return params, losses

# 6. Обучение отдельных моделей для площади и периметра
print("\nОбучение модели для ПЛОЩАДИ:")
params_area, losses_area = train_model(X_poly, area, n_epochs=2000, lr=0.01)

print("\nОбучение модели для ПЕРИМЕТРА:")
params_perim, losses_perim = train_model(X_poly, perimeter, n_epochs=2000, lr=0.01)

# 7. Предсказание на новых данных
print("\n" + "="*50)
print("ПРЕДСКАЗАНИЕ НА НОВЫХ ДАННЫХ")

# Новые данные (длина, ширина)
new_data = np.array([
    [3.5, 2.8],  # Пример 1
    [7.2, 4.1],  # Пример 2
    [2.0, 5.5]   # Пример 3
])

# Создаем полиномиальные признаки для новых данных
new_data_poly = create_poly_features(new_data, degree=3)

# Делаем предсказания
area_predictions = np.dot(new_data_poly, params_area)
perim_predictions = np.dot(new_data_poly, params_perim)

# Выводим результаты
print("\nНовые прямоугольники:")
for i, (l, w) in enumerate(new_data):
    true_area = l * w
    true_perim = 2 * (l + w)

    print(f"\nПрямоугольник {i+1}: длина={l:.1f}, ширина={w:.1f}")
    print(f"  Истинная площадь: {true_area:.2f}, Предсказанная: {area_predictions[i]:.2f}")
    print(f"  Истинный периметр: {true_perim:.2f}, Предсказанный: {perim_predictions[i]:.2f}")
    print(f"  Ошибка площади: {abs(true_area - area_predictions[i]):.2f}")
    print(f"  Ошибка периметра: {abs(true_perim - perim_predictions[i]):.2f}")

# 8. Проверка на обучающих данных (опционально)
print("\n" + "="*50)
print("ПРОВЕРКА НА ОБУЧАЮЩИХ ДАННЫХ:")
train_area_pred = np.dot(X_poly, params_area)
train_perim_pred = np.dot(X_poly, params_perim)

print(f"Ошибка на обучающих данных (площадь): {compute_loss(area, train_area_pred):.6f}")
print(f"Ошибка на обучающих данных (периметр): {compute_loss(perimeter, train_perim_pred):.6f}")

Обучающие данные (длина, ширина):
[[4.37086107 1.28286267]
 [9.55642876 6.7276937 ]
 [7.58794548 3.82920383]
 [6.38792636 5.57713622]
 [2.40416776 9.16809827]
 [2.40395068 3.24363006]
 [1.52275251 4.69344631]
 [8.79558531 7.79996025]
 [6.41003511 3.05918349]
 [7.3726532  1.69281919]
 [1.18526045 3.60776308]
 [9.72918867 2.45099159]
 [8.49198377 9.36727887]
 [2.911052   8.27308342]
 [2.6364247  6.70063381]
 [2.65064059 8.84314531]
 [3.73818019 8.23304869]
 [5.72280788 2.67913053]
 [4.88750517 9.03303099]
 [3.62106226 5.85408018]
 [6.50667605 8.2669614 ]
 [2.25544475 9.0648217 ]
 [3.62930184 3.86203127]
 [4.29725659 1.99046732]
 [5.10462986 3.05141646]
 [8.06658365 4.8439701 ]
 [2.79706404 8.36213289]
 [5.62810995 8.74657525]
 [6.33173112 1.06256917]
 [1.41805371 5.59672572]
 [6.46790367 4.75669903]
 [2.53471711 2.99897029]
 [1.58546434 2.07878831]
 [9.53996984 4.03853654]
 [9.6906883  9.48618734]
 [8.27557613 3.90882639]
 [3.74152392 5.6691156 ]
 [1.87904903 7.32717063]
 [7.15809724 4.2